# Extracting information from input in structured manner

This notebook performs simple sentiment analysis of input text, returning function arguments that contain some analysis of the input.

The usual boiler plate to import libraries and set the model
You'll need your OpenAI key set as environment variable `OPENAI_API_KEY`

In [8]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4"

This is is the usual function to make a call to ChatGPT API completetion endpoint, taking a list of messages 
and optionally a list of functions, and optionally an instruction to create arguments for a specified function

In [9]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:T
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [42]:
## We could send this to logs or analytics if we wanted to track sentiment over time
## e.g. for analysing feedback
def sentiment (sentiment="", initialInput="", sentimentScore=0, algorithm="chatgpt magic"):
    print(f'sentiment of "{initialInput}" is "{sentiment}" with score {sentimentScore} using algorithm {algorithm}')

## a dictionary of functions keyed by their name. 
available_functions = {
    "sentiment":sentiment, 
}

## The function definitions we will send to ChatGPT. The 'parameters' object is defined using JSON Schema.
functions = [
    {
        "name" : "sentiment",
        "description": "Obtain the sentiment for the given input and supply as function argument, along with the input,  a score between -1 and 1 and a short description of the algorithm used ",
        "parameters" : {
            "type": "object",
            "properties" : {
                "sentiment": {
                     "type":"string",
                     "enum": ['Positive', 'Neutral', 'Negative']
                },
                "sentimentScore": {
                     "type":"number",
                },
                "initialInput": {
                    "type":"string"
                },
                "algorithm": {
                    "type":"string"
                }
            }
        }
    }
]
        

In [36]:
inputs = [
    "Oh no, another KenWhat meeting to endure",
    "Great! another  KenWhat meeting to savour",
    "I am going to listen to two hours of talks with some modest degree of enthusiasm"
]

In [18]:
messages = [
 {
     "role":"system",
     "content":"Analyse user input returning function arguments"
 },
 {
     "role": "user",
     "content":"Replace me"
 }
]

In [44]:
for input in inputs:
    messages[1]['content'] = input
    resp = chat_completion_request(messages, functions, function_call={'name':'sentiment'})
    
    response_message = resp.json()['choices'][0]['message']
    print(response_message)
    
    if 'function_call' in response_message and response_message['function_call'] is not None:
        to_call = response_message['function_call']['name']
        f_args = json.loads(response_message['function_call']['arguments'])
        result = available_functions[to_call](**f_args)

    else:
        print(f'no function was returned - {response_message["content"]}')
    

{'role': 'assistant', 'content': None, 'function_call': {'name': 'sentiment', 'arguments': '{\n  "sentiment": "Negative",\n  "sentimentScore": -0.6,\n  "initialInput": "Oh no, another KenWhat meeting to endure",\n  "algorithm": "Natural Language Processing and Machine Learning Algorithms"\n}'}}
sentiment of "Oh no, another KenWhat meeting to endure" is "Negative" with score -0.6 using algorithm Natural Language Processing and Machine Learning Algorithms
{'role': 'assistant', 'content': None, 'function_call': {'name': 'sentiment', 'arguments': '{\n  "sentiment": "Positive",\n  "sentimentScore": 0.9,\n  "initialInput": "Great! another  KenWhat meeting to savour",\n  "algorithm": "The sentiment was derived from analyzing the positive connotation in words such as \'Great\' and \'savour\'."\n}'}}
sentiment of "Great! another  KenWhat meeting to savour" is "Positive" with score 0.9 using algorithm The sentiment was derived from analyzing the positive connotation in words such as 'Great' and 